In [1]:
import mirdata
print(mirdata.list_datasets())
maestro = mirdata.initialize('maestro', data_home='/root/musica/data')

['acousticbrainz_genre', 'baf', 'ballroom', 'beatles', 'beatport_key', 'billboard', 'brid', 'candombe', 'cante100', 'cipi', 'compmusic_carnatic_rhythm', 'compmusic_carnatic_varnam', 'compmusic_hindustani_rhythm', 'compmusic_indian_tonic', 'compmusic_jingju_acappella', 'compmusic_otmm_makam', 'compmusic_raga', 'cuidado', 'da_tacos', 'dagstuhl_choirset', 'dali', 'egfxset', 'filosax', 'four_way_tabla', 'freesound_one_shot_percussive_sounds', 'giantsteps_key', 'giantsteps_tempo', 'good_sounds', 'groove_midi', 'gtzan_genre', 'guitarset', 'hainsworth', 'haydn_op20', 'idmt_smt_audio_effects', 'ikala', 'irmas', 'maestro', 'mdb_stem_synth', 'medley_solos_db', 'medleydb_melody', 'medleydb_pitch', 'mridangam_stroke', 'mtg_jamendo_autotagging_moodtheme', 'openmic2018', 'orchset', 'phenicx_anechoic', 'queen', 'rwc_classical', 'rwc_jazz', 'rwc_popular', 'salami', 'saraga_carnatic', 'saraga_hindustani', 'scms', 'simac', 'slakh', 'tinysol', 'tonality_classicaldb', 'tonas', 'vocadito']


INFO: Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO: NumExpr defaulting to 16 threads.


In [2]:
# maestro.download()

In [3]:
# maestro.download(partial_download=['index'])

In [4]:
example_track = maestro.choice_track()

In [5]:
example_track

Track(
  audio_path="...oot/musica/data/2004/MIDI-Unprocessed_XP_20_R2_2004_01_ORIG_MID--AUDIO_20_R1_2004_01_Track01_wav.wav",
  midi_path="...ot/musica/data/2004/MIDI-Unprocessed_XP_20_R2_2004_01_ORIG_MID--AUDIO_20_R1_2004_01_Track01_wav.midi",
  track_id="2004/MIDI-Unprocessed_XP_20_R2_2004_01_ORIG_MID--AUDIO_20_R1_2004_01_Track01_wav",
  audio: The track's audio

        Returns,
  canonical_composer: ,
  canonical_title: ,
  duration: ,
  midi: ,
  notes: ,
  split: ,
  year: ,
)

In [6]:
midi_path = '/root/musica/data/2008/MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--1.midi'
wav_path = '/root/musica/data/2008/MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--1.wav'

In [15]:
import torchaudio
example_melody = example_track.audio_path

# Load audio
waveform, sample_rate = torchaudio.load(example_track.audio_path)

In [7]:
import mido

# Load the MIDI file
midi_file = mido.MidiFile(midi_path)

# Iterate through the messages in the file
for i, track in enumerate(midi_file.tracks):
    print(f"Track {i}: {track.name}")
    for msg in track:
        print(msg)

Track 0: 
MetaMessage('set_tempo', tempo=500000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('end_of_track', time=1)
Track 1: Cui05-08.mid
MetaMessage('track_name', name='Cui05-08.mid', time=0)
program_change channel=0 program=0 time=0
control_change channel=0 control=67 value=11 time=370
control_change channel=0 control=67 value=22 time=15
control_change channel=0 control=67 value=36 time=15
control_change channel=0 control=67 value=51 time=16
control_change channel=0 control=67 value=64 time=14
control_change channel=0 control=67 value=79 time=15
control_change channel=0 control=67 value=90 time=16
control_change channel=0 control=67 value=101 time=14
control_change channel=0 control=67 value=110 time=16
control_change channel=0 control=67 value=118 time=15
control_change channel=0 control=67 value=126 time=15
control_change channel=0 control=67 value=127 time=16
note_on channel=0 note=51 ve

In [78]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

sample_rate = 44.1e3
n_fft = 2048
hop_length = 128
n_mels = 256
y, sr = librosa.load(wav_path, sr = None, mono=True)
# sr = 16000 # Hz
target_sr = 16e3
y = librosa.resample(y, orig_sr=sample_rate, target_sr=target_sr)

# Compute mel spectrogram
S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft = n_fft, hop_length=hop_length, n_mels=n_mels, fmax=8000)

# Convert to decibels (log scale)
S_dB = librosa.power_to_db(S, ref=np.max)

In [51]:
# y, sr = librosa.load(wav_path, sr = sample_rate)
duration = librosa.get_duration(y=y, sr=sr)
duration

343.97748299319727

In [54]:
import soundfile as sf

info = sf.info(wav_path)
print(info)

/root/musica/data/2008/MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--1.wav
samplerate: 44100 Hz
channels: 2
duration: 05:43.977 min
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]


In [47]:
# Plot it
plt.figure(figsize=(10, 4))
librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel', fmax=8000)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram')
plt.tight_layout()
plt.show()

KeyboardInterrupt: 

In [26]:
S_dB.shape

(256, 42998)

In [27]:
42998/sr

2.687375

In [19]:
sr

44100.0

In [83]:
for i, track in enumerate(midi_file.tracks):
    print(f"Track {i}: {track.name}")
    for msg in track:
        print(msg)


Track 0: 
MetaMessage('set_tempo', tempo=500000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('end_of_track', time=1)
Track 1: Cui05-08.mid
MetaMessage('track_name', name='Cui05-08.mid', time=0)
program_change channel=0 program=0 time=0
control_change channel=0 control=67 value=11 time=370
control_change channel=0 control=67 value=22 time=15
control_change channel=0 control=67 value=36 time=15
control_change channel=0 control=67 value=51 time=16
control_change channel=0 control=67 value=64 time=14
control_change channel=0 control=67 value=79 time=15
control_change channel=0 control=67 value=90 time=16
control_change channel=0 control=67 value=101 time=14
control_change channel=0 control=67 value=110 time=16
control_change channel=0 control=67 value=118 time=15
control_change channel=0 control=67 value=126 time=15
control_change channel=0 control=67 value=127 time=16
note_on channel=0 note=51 ve

In [92]:
midi_file.tracks[1][10].type

'control_change'

In [ ]:
from miditok import MIDILike, TokenizerConfig
from symusic import Score

tokenizer = MIDILike(TokenizerConfig())

# Creating a multitrack tokenizer, read the doc to explore all the parameters
config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True)
tokenizer = MIDILike(config)

# Loads a midi, converts to tokens, and back to a MIDI
midi = Score(example_track.midi_path)
tokens = tokenizer(midi)  # calling the tokenizer will automatically detect MIDIs, paths and tokens
converted_back_midi = tokenizer(tokens)  

/tmp/ipykernel_9199/1623911266.py:7: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  tokenizer = MIDILike(config)


In [117]:
class Tokeniser:
    def __init__(self, ):
        return
    
    @staticmethod
    def process_midi(midi):
        # In: Librosa midi file
        times = []
        msgs = []
        for i, track in enumerate(midi.tracks):
            abs_time = 0

            # remove initial track
            if i ==0:
                continue

            for msg in track:
                abs_time += msg.time
                if msg.type in ['note_on', 'note_off']:
                    msgs.append(msg)
                    times.append(abs_time)
                else:
                    continue

        times=  torch.tensor(times)
        return times, msgs
    
    @staticmethod
    def collect_indices_between(times, start_time, end_time):
        return [i for i, t in enumerate(times) if start_time <= t <= end_time] 

    @staticmethod
    def get_midi_between(midi, start_time, end_time):
        times, msgs = Tokeniser.process_midi(midi)
        idxs = Tokeniser.collect_indices_between(times, start_time, end_time)
        return times[idxs], msgs[idxs[0]:idxs[-1]]
    
tok = Tokeniser()
times, msgs = tok.get_midi_between(midi_file, 10000, 20000)
times, msgs

(tensor([10093, 10122, 10332, 10337, 10530, 10568, 10797, 10949, 11109, 11110,
         11297, 11315, 11493, 11500, 11685, 11706, 11866, 11883, 12019, 12079,
         12178, 12301, 12334, 12599, 12601, 12654, 12762, 12908, 13107, 13119,
         13294, 13321, 13504, 13522, 13697, 13709, 13873, 13904, 14085, 14118,
         14346, 14386, 14564, 14685, 14710, 14716, 14768, 14886, 14923, 14934,
         14944, 15100, 15123, 15128, 15161, 15291, 15314, 15317, 15354, 15500,
         15508, 15508, 15535, 15691, 15705, 15708, 15721, 15896, 15905, 15906,
         15913, 16094, 16120, 16194, 16388, 16389, 16593, 16596, 16776, 16789,
         16984, 17020, 17183, 17206, 17290, 17379, 17384, 17580, 17593, 17819,
         17824, 17827, 17982, 18077, 18082, 18274, 18285, 18285, 18286, 18310,
         18461, 18481, 18484, 18512, 18653, 18674, 18684, 18736, 18858, 18870,
         18876, 18882, 18936, 19068, 19069, 19099, 19141, 19275, 19288, 19289,
         19329, 19525, 19526, 19563, 19576, 19816, 1

In [104]:
import pandas as pd
df = pd.read_csv('/root/musica/data/maestro-v2.0.0.csv')
df = df[df['year'].isin([2008, 2009, 2011, 2014])]

In [ ]:
from torch.utils.data import Dataset
import os
import torch 

class MaestroDataset(Dataset):
    def __init__(self, df_root):
        self.dir = df_root
        self.df = pd.read_csv(os.path.join(df_root, 'maestro-v2.0.0.csv'))
        self.df = df[df['year'].isin([2008, 2009])]

        self.n_fft = 2048
        self.hop_length = 128
        self.n_mels = 256
        self.target_sr = 16e3

    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        midi_filename = os.path.join(df_root, self.df.iloc[index]['midi_filename'])
        audio_filename = os.path.join(df_root, self.df.iloc[index]['audio_filename'])

        y, sr = librosa.load(audio_filename, sr = None, mono=True)
        # resample to lower rate
        y = librosa.resample(y, orig_sr=sample_rate, target_sr=target_sr)

        # Compute mel spectrogram
        S = librosa.feature.melspectrogram(y=y, sr=sr, 
                                           n_fft = self.n_fft, 
                                           hop_length=self.hop_length, 
                                           n_mels=self.n_mels, fmax=8000)

        # Convert to decibels (log scale)
        S_dB = librosa.power_to_db(S, ref=np.max)

        return torch.tensor(S_dB), midi_filename
    
df_root = '/root/musica/data' 
maestro = MaestroDataset(df_root)
maestro[0]

(tensor([[-54.6900, -52.5696, -51.3081,  ..., -51.3898, -53.0736, -55.7320],
         [-47.6586, -45.5382, -44.2767,  ..., -44.3584, -46.0422, -48.7006],
         [-52.8492, -50.6202, -49.1364,  ..., -47.3676, -50.1173, -53.9425],
         ...,
         [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -80.0000],
         [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -80.0000],
         [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -80.0000]]),
 '/root/musica/data/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi')

In [76]:
maestro.df

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
5,Alexander Scriabin,"5 Preludes, Op.15",validation,2009,2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MI...,2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MI...,400.557826
6,Alexander Scriabin,"Entragete, Op.63",test,2009,2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MI...,2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MI...,163.745830
8,Alexander Scriabin,"Etude Op. 42, Nos. 4 & 5",test,2009,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MI...,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MI...,136.315302
9,Alexander Scriabin,"Etude Op. 8, No. 13",validation,2009,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MI...,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MI...,167.085837
...,...,...,...,...,...,...,...
1264,Wolfgang Amadeus Mozart,"Sonata in D Major, K. 284, First Movement",train,2011,2011/MIDI-Unprocessed_03_R1_2011_MID--AUDIO_R1...,2011/MIDI-Unprocessed_03_R1_2011_MID--AUDIO_R1...,205.236173
1265,Wolfgang Amadeus Mozart,"Sonata in D Major, K. 311",train,2009,2009/MIDI-Unprocessed_14_R1_2009_06-08_ORIG_MI...,2009/MIDI-Unprocessed_14_R1_2009_06-08_ORIG_MI...,910.278203
1266,Wolfgang Amadeus Mozart,"Sonata in D Major, K. 576 (Complete)",train,2014,2014/MIDI-UNPROCESSED_09-10_R1_2014_MID--AUDIO...,2014/MIDI-UNPROCESSED_09-10_R1_2014_MID--AUDIO...,790.820523
1269,Wolfgang Amadeus Mozart,"Sonata in D Major, K.311",train,2009,2009/MIDI-Unprocessed_05_R1_2009_01-02_ORIG_MI...,2009/MIDI-Unprocessed_05_R1_2009_01-02_ORIG_MI...,792.420415
